In [ ]:
pip install fosforml

     |████████████████████████████████| 40kB 598kB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 1.2MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 41.0MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 69.5MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 94.9MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 108.5MB/s eta 0:00:01
     |▏                               | 921kB 61.0MB/s eta 0:00:04

In [ ]:
pip install seaborn

In [ ]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

In [ ]:
pip install ydata-profiling --upgrade

In [ ]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
import seaborn as sns

In [ ]:
my_session.connection.database

In [ ]:
my_session.connection.schema

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
df=sf_df.to_pandas()

In [ ]:
type(df)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['TRANS_DATE'] = pd.to_datetime(df['TRANS_DATE'])
df['START_DATE'] = pd.to_datetime(df['START_DATE'])

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.corr(numeric_only=True)

In [ ]:
from ydata_profiling import ProfileReport
profile=ProfileReport(df,explorative=True)
profile.to_file("autoeda.html")